# **PYED+CPT **:Cluster pertrubation theory with exact diagonalization for finite quantum systems

Copyright (C) 2017, H. U.R. Strand, Ya.V. Zhumagulov

The python module `pyed` implements exact diagonalization for finite fermionic many-body quantum systems, together with calculations of several response functions in imagianary time.

The many-body system is defined using `pytriqs` second-quantized operators and the response functions are stored in `pytriqs` Green's function containters.

Cluster pertrubation theory addition to `pyed`  allow calculate bandstructure and Fermi surface of several models. 

## Hamiltonians

 As an example let us solve the Hubbard model with Hamiltonian including only nearest-neighbor hoppings $H = U\sum_{i}\hat{n}_{i,\uparrow} \hat{n}_{i,\downarrow} - \mu\sum_{i}( \hat{n}_{i,\uparrow} + \hat{n}_{i,\downarrow}) + t \sum_{<ij>,\sigma}c^\dagger_{i,\sigma} c_{j\sigma}$, where $\hat{n}_\sigma = c^\dagger_\sigma c_\sigma$. 


In [18]:
from pytriqs.operators import c, c_dag,n
from pyed.TriqsExactDiagonalization import TriqsExactDiagonalization
from pyed.ClusterPertrubationTheory import ClusterPertrubationTheory_2D_Square
import numpy as np
import progressbar
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline

Parameters of model: U=8, t=-1, $\mu$=U/2 and size of exact diagonaliztion cluster will be 2x2

In [30]:
t =-1;U=8;mu=U/2
Lx,Ly=2,2;L=Lx*Ly

In [31]:
T=np.zeros((L,L))
for i in range(L):
    for j in range(L):
        x = i % Lx - j % Ly
        y = i // Lx - j // Ly
        if (x**2+y**2)==1: T[i,j]=t            
H_int = sum(-mu*(n('up', site) + n('dn', site)) + U * n('up', site) * n('dn', site) for site in range(L))
H_kin = sum(T[st1][st2]*c_dag(sn,st1)*c(sn,st2) for sn, st1,st2 in product(("dn", "up"), range(L),range(L)) )
H = H_int +H_kin

Parameter $\beta$ will be 200

In [32]:
fundamental_operators = np.array([[c('up',i), c('dn',i)] for i in range(L)]).flatten()
H=H_int+H_kin
beta=200;nstates=200
ed = TriqsExactDiagonalization(H,fundamental_operators, beta,nstates=nstates)

Hamiltonian diagonalization:


100% |########################################################################|


Now we construct, frequency meshgrid, momentum meshgrid and pertrubation matrix V

Further explation you can find in:

https://www.physique.usherbrooke.ca/pages/sites/default/files/senechal/publis/Senechal2011vn.pdf

In [37]:
N=60
kx = np.linspace(-np.pi, np.pi, N+1);kx = np.delete(kx, 0)
ky = np.linspace(-np.pi, np.pi,N+1);ky = np.delete(ky, 0)
kx, ky = np.meshgrid(kx, ky)
V=np.zeros((N,N,L,L),dtype=np.complex)
for a,b in product(range(L),range(L)):
        x=a % Lx - b % Ly ; y=a// Lx - b// Ly
        if (y==(Ly-1))&(x==0):V[:,:,a,b]=t*np.exp(1j*Ly*ky);V[:,:,b,a]=t*np.exp(-1j*Ly*ky)
        if (x==(Lx-1))&(y==0):V[:,:,a,b]=t*np.exp(1j*Lx*kx);V[:,:,b,a]=t*np.exp(-1j*Lx*kx)

To apply Cluster Pertrubation Theory to our exact diagonalization we use `ClusterPertrubationTheory_2D_Square` class for 2D Square models

In [ ]:
omega=np.linspace(-10,10,200);
CPT=ClusterPertrubationTheory_2D_Square(ed,(kx,ky),V,omega,(Lx,Ly))

Calculation green function of full system


100% |########################################################################|


Coupling system


Now we can calculate Fermi surface

In [ ]:
plt.figure(figsize=(6,6))
CPT.calculation_Fermi_surface()
plt.imshow(CPT.FS,interpolation='lanczos')

And band structure

In [ ]:
plt.figure(figsize=(12,6))
CPT.calculation_bandstructure()
plt.imshow(CPT.bandstructure,cmap=plt.cm.jet,interpolation='lanczos', aspect='auto')